<a href="https://colab.research.google.com/github/0906Bao/TriTueNhanTao/blob/main/Tuan03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Thuật toán Minimax**

In [1]:
import copy
import math
import random
import numpy

X = "X"
O = "O"
EMPTY = None
user = None
ai = None

In [2]:
def initial_state():
  return [[EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY]]

Khởi tạo trạng thái bắt đầu của bàn cờ Tic-Tac-Toe. Nó trả về một danh sách lồng nhau (ma trận 3x3), trong đó tất cả các ô đều có giá trị là EMPTY (trống), biểu thị rằng chưa có nước đi nào được thực hiện.

In [3]:
def player(board):
  count = 0
  for i in board:
    for j in i:
      if j:
        count += 1
  if count % 2 != 0:
    return ai
  return user

Hàm này xác định xem hiện tại đang là lượt đi của ai. Thực hiện bằng cách đếm số lượng ô đã được đánh dấu trên bàn cờ. Trong luật chơi thông thường, X đi trước, O đi sau. Do đó, nếu số lượng nước đi là số lẻ, hàm trả về lượt của AI, ngược lại trả về lượt của user.

In [4]:
def actions(board):
  res = set()
  board_len = len(board)
  for i in range(board_len):
    for j in range(board_len):
      if board[i][j] == EMPTY:
        res.add((i, j))
  return res

Hàm này tìm và trả về tất cả các nước đi hợp lệ có thể thực hiện được từ trạng thái bàn cờ hiện tại. Duyệt qua từng ô trên bàn cờ 3x3, nếu ô đó còn trống (EMPTY), nó sẽ thêm tọa độ (hàng, cột) của ô đó vào một tập hợp (set) kết quả.

In [5]:
def result(board, action):
  curr_player = player(board)
  result_board = copy.deepcopy(board)
  (i, j) = action
  result_board[i][j] = curr_player
  return result_board

Hàm này mô phỏng kết quả của một nước đi. Nó nhận vào trạng thái bàn cờ hiện tại và một hành động. Đầu tiên, nó xác định người chơi hiện tại, sau đó tạo ra một bản sao (deepcopy) của bàn cờ để không làm ảnh hưởng đến bàn cờ chính. Cuối cùng, nó đánh dấu nước đi vào bản sao đó và trả về trạng thái bàn cờ mới.

In [6]:
def get_horizontal_winner(board):
  winner_val = None
  board_len = len(board)
  for i in range(board_len):
    winner_val = board[i][0]
    for j in range(board_len):
      if board [i][j] != winner_val:
        winner_val = None
      if winner_val:
        return winner_val
  return winner_val

Hàm này kiểm tra xem có người chiến thắng theo hàng ngang hay không. Nó duyệt qua từng hàng, giả định phần tử đầu tiên của hàng là người thắng, sau đó so sánh với các phần tử còn lại trong hàng đó. Nếu tất cả giống nhau và không phải là ô trống, nó trả về người chiến thắng.

In [7]:
def get_vertical_winner(board):
  winner_val = None
  board_len = len(board)
  for i in range(board_len):
    winner_val = board[0][i]
    for j in range(board_len):
      if board[j][i] != winner_val:
        winner_val = None
    if winner_val:
      return winner_val
  return winner_val

Hàm này tương tự như hàm kiểm tra hàng ngang, nhưng nó kiểm tra chiến thắng theo hàng dọc (cột). Nó duyệt qua từng cột và kiểm tra xem các ô trong cột đó có giống nhau và khác rỗng hay không.

In [8]:
def get_diagonal_winner(board):
  winner_val = None
  board_len = len(board)
  winner_val = board[0][0]
  for i in range(board_len):
    if board[i][i] != winner_val:
      winner_val = None
  if winner_val:
    return winner_val
  winner_val = board[0][board_len - 1]
  for i in range(board_len):
    j = board_len - 1 - i
    if board[i][j] != winner_val:
      winner_val = None
  return winner_val

Hàm này kiểm tra chiến thắng theo hai đường chéo chính và phụ. Đầu tiên nó kiểm tra đường chéo từ góc trên-trái xuống góc dưới-phải. Sau đó, nó kiểm tra đường chéo từ góc trên-phải xuống góc dưới-trái.

In [9]:
def winner(board):
  winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
  return winner_val

Hàm tổng hợp để xác định người chiến thắng cuối cùng. Nó gọi lần lượt các hàm kiểm tra hàng ngang, hàng dọc và đường chéo. Nếu bất kỳ hàm nào trả về người thắng, hàm này sẽ trả về kết quả đó. Nếu không ai thắng, trả về None.

In [10]:
def terminal(board):
  if winner(board) != None:
    return True
  for i in board:
    for j in i:
      if j == EMPTY:
        return False
  return True

Hàm này kiểm tra xem trò chơi đã kết thúc chưa. Trò chơi kết thúc khi đã có người thắng (hàm winner trả về giá trị khác None) hoặc khi bàn cờ đã đầy (không còn ô EMPTY nào).

In [11]:
def utility(board):
  winner_val = winner(board)
  if winner_val == X:
    return 1
  elif winner_val == O:
    return -1
  return 0

Hàm gán giá trị số cho kết quả của bàn cờ. Nếu X thắng được gán giá trị 1, nếu O thắng được gán giá trị -1, và nếu hòa thì giá trị là 0.

In [12]:
def maxValue(state):
  if terminal(state):
    return utility(state)
  v = -math.inf
  for action in actions(state):
    v = min(v, minValue(result (state, action)))
  return v

Hàm này đại diện cho lượt chơi của người chơi max (thường là X). Nếu trạng thái là kết thúc, nó trả về điểm số. Nếu không, nó khởi tạo giá trị v là âm vô cùng, sau đó duyệt qua các hành động có thể để tìm nước đi mang lại giá trị lớn nhất.

In [13]:
def minValue(state):
  if terminal(state):
    return utility(state)
  v = math.inf
  for action in actions(state):
    v = min(v, maxValue(result (state, action)))
  return v

Hàm này đại diện cho lượt chơi của người chơi min (thường là O). Nó khởi tạo giá trị v là dương vô cùng và duyệt qua các hành động để tìm nước đi dẫn đến giá trị nhỏ nhất (có lợi cho O).

In [14]:
def minimax(board):
  current_player = player(board)
  if current_player == X:
    for action in actions(board):
      check = minValue(result(board, action))
      if check > min:
        min_value = check
        move = action
  else:
    max_value = math.inf
    for action in actions(board):
      check = maxValue(result(board, action))
      if check < max_value:
        max_value = check
        move = action
  return move

Đây là hàm chính điều khiển thuật toán Minimax để chọn nước đi tối ưu. Nó xác định người chơi hiện tại. Nếu là X, nó sẽ thử tất cả các nước đi và chọn nước đi có giá trị trả về từ minValue lớn nhất. Nếu là O, nó sẽ chọn nước đi có giá trị trả về từ maxValue nhỏ nhất.

In [ ]:
if __name__ == "__main__":
  board = initial_state()
  ai_turn = False
  print("Choose a player")
  user = input()
  if user == "X":
    ai = "O"
  else:
    ai = "X"
  while True:
    game_over = terminal(board)
    playr = player(board)
    if game_over:
      winner = winner(board)
      if winner is None:
        print("game over: tie")
      else:
        print(f"game over: {winner} wins.")
      break
    else:
      if user != playr and not game_over:
        if ai_turn:
          move = minimax(board)
          board = result(board, move)
          ai_turn = False
          print(numpy.array(board))
        elif user == playr and not game_over:
          ai_turn = True
          print("enter the position to move (row, col)")
          i = int(input("row: "))
          j = int(input("col: "))
          if board[i][j] == EMPTY:
            board = result(board, (i, j))
            print(numpy.array(board))

**Thuật toán cắt tỉa Alpha-beta**

In [17]:
import os, math

In [18]:
def GetWinner(board):
  if board[0] == board[1] and board[1] == board[2]:
    return board[0]
  elif board[3] == board[4] and board[4] == board[5]:
    return board[3]
  elif board[6] == board[7] and board[7] == board[8]:
    return board[6]
  elif board[0] == board[3] and board[3] == board[6]:
    return board[0]
  elif board[1] == board[4] and board[4] == board[7]:
    return board[1]
  elif board[2] == board[5] and board[5] == board[8]:
    return board[2]
  elif board[0] == board[4] and board[4] == board[8]:
    return board[0]
  elif board[2] == board[4] and board[4] == board[6]:
    return board[2]

Hàm này kiểm tra người chiến thắng trên bàn cờ. So sánh trực tiếp các chỉ số tương ứng với 8 trường hợp thắng (3 hàng ngang, 3 hàng dọc, 2 đường chéo) để xem các ký tự có giống nhau hay không và trả về ký tự thắng cuộc.

In [19]:
def PrintBoard(board):
  os.system('cls' if os.name=='nt' else 'clear')
  print(f'''
  {board[0]}|{board[1]}|{board[2]}
  {board[3]}|{board[4]}|{board[5]}
  {board[6]}|{board[7]}|{board[8]}
  ''')

Hàm này dùng để hiển thị bàn cờ ra màn hình console. Trước khi in, nó xóa màn hình (lệnh cls trên Windows hoặc clear trên Linux/Mac) để giao diện trông gọn hơn, sau đó in lưới 3x3 với các giá trị hiện tại của bàn cờ.

In [20]:
def GetAvailableCells(board):
  available = list()
  for cell in board:
    if cell != "X" and cell != "O":
      available.append(cell)
  return available

Hàm này nhằm mục đích tìm các ô trống. Trong code này, bàn cờ được khởi tạo bằng các số từ 1-9. Nếu một ô không phải là "X" hay "O", nghĩa là nó chưa được đánh.

In [21]:
def minimax(position, depth, alpha, beta, isMaximizing):
  winner = GetWinner(position)
  if winner != None:
    return 10 - depth if winner == "X" else -10 + depth
  if len(GetAvailableCells(position)) == 0:
    return 0

  if isMaximizing:
    maxEval = -math.inf
    for cell in GetAvailableCells(position):
      position[cell - 1] = "X"
      Eval = minimax(position, depth + 1, alpha, beta, False)
      maxEval = max(maxEval, Eval)
      alpha = max(alpha, Eval)
      position[cell - 1] = cell
      if beta <= alpha:
        break
    return maxEval
  else:
    minEval = math.inf
    for cell in GetAvailableCells(position):
      position[cell - 1] = "O"
      Eval = minimax(position, depth + 1, alpha, beta, True)
      minEval = min(minEval, Eval)
      beta = min(beta, Eval)
      position[cell - 1] = cell
      if beta <= alpha:
        break
    return minEval

Đây là hàm đệ quy thực hiện thuật toán Minimax kết hợp cắt tỉa Alpha-beta.


*   Nếu có người thắng, nó trả về điểm số (cộng/trừ depth để ưu tiên thắng nhanh hoặc thua chậm).
*   Nếu isMaximizing (lượt của X): Nó tìm nước đi có điểm cao nhất, cập nhật giá trị alpha. Nếu beta <= alpha, nó cắt tỉa (dừng xét nhánh này) vì đối thủ sẽ không bao giờ cho phép nhánh này xảy ra.
*   Nếu không phải isMaximizing (lượt của O): Nó tìm nước đi có điểm thấp nhất, cập nhật giá trị beta. Cắt tỉa nếu beta <= alpha.



In [22]:
def FindBestMove (CurrentPosition, AI):
  bestVal = -math.inf if AI == "X" else math.inf
  bestMove = -1
  for cell in GetAvailableCells(CurrentPosition):
    CurrentPosition[cell - 1] = AI
    moveVal = minimax(CurrentPosition, 0, -math.inf, math.inf, False if AI == "X" else True)
    CurrentPosition[cell - 1] = cell
    if (AI == "X" and moveVal > bestVal):
      bestMove = cell
      bestVal = moveVal
    elif (AI == "O" and moveVal < bestVal):
      bestMove = cell
      bestVal = moveVal
  return bestMove


Hàm này là điểm bắt đầu để AI tìm nước đi tốt nhất. Nó duyệt qua tất cả các ô trống hiện có, thử đi vào ô đó, rồi gọi hàm minimax để tính điểm cho nước đi đó. Sau khi thử xong, nó hoàn tác nước đi và chọn ra ô có điểm số tốt nhất để thực hiện.

In [23]:
def main():
  player = input("play as X or O? ").strip().upper()
  AI = "O" if player == "X" else "X"
  currentGame = [*range(1, 10)]
  currentTurn = "X"
  counter = 0
  while True:
    if currentTurn == AI:
      cell = FindBestMove(currentGame, AI)
      currentGame[cell - 1] = AI
      currentTurn = player
    elif currentTurn == player:
      PrintBoard(currentGame)
      while 1:
        humanInput = int(input("Enter Number: ").strip())
        if humanInput in currentGame:
          currentGame[humanInput - 1] = player
          currentTurn = AI
          break
        else:
          PrintBoard(currentGame)
          print("Cell Not Available.")
    if GetWinner(currentGame) != None:
      PrintBoard(currentGame)
      print(f"{GetWinner(currentGame)} Won!!!")
      break
    counter += 1
    if GetWinner(currentGame) == None and counter == 9:
      PrintBoard(currentGame)
      print("Tie.")
      break


In [24]:

if __name__ == "__main__":
  main()

play as X or O? X

  1|2|3
  4|5|6
  7|8|9
  
Enter Number: 1

  X|2|3
  4|O|6
  7|8|9
  
Enter Number: 2

  X|X|O
  4|O|6
  7|8|9
  
Enter Number: 6

  X|X|O
  4|O|X
  O|8|9
  
O Won!!!
